# Load Required Libraries

In [1]:
import pkg_resources
import requests
import json
import time
from datetime import datetime

# Load libraries
import numpy as np
import pandas as pd
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
from datetime import date
from sklearn import cluster, covariance, manifold
from sklearn.linear_model import LinearRegression

from scipy.stats import pearsonr

# Required Inputs

Right now there are a few manual inputs needed to do the calculations. They are:

- The historical allocation of the CS portfolio.  Currently, I have a spreadsheet that has the allocations which I periodically upload to github and connect to my notebook.  Maintaining the spreadsheet is easy because the allocations only change when we make transactions (which is rare). There is probably an easier way to do this but I haven't worked it out yet (possibility of using Binance APIs: https://binance-docs.github.io/apidocs/spot/en/#daily-account-snapshot-user_data)


- We need a list of the assets in the portfolio.

In [2]:
#Link to Github Raw Data of CS Portfolio Allocation
cs_weights_csv = 'https://raw.githubusercontent.com/kinji20/Most_Recent_Token_Price/main/LTG_Assets%20-%20Invested_Capital%20(2).csv'

#List of asset symbols as a string
assets = ['BTC', 'ETH', 'SOL', 'UNI', 'AR', 'DOT', 'AVAX', 'YFI', 'LINK', 'CRV', 'LUNA', 'AAVE']

In [26]:
allocations_df

,BTC,ETH,SOL,UNI,AR,DOT,AVAX,YFI,LINK,CRV,LUNA,AAVE,Total Investment (USD),timestamp
0,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,23.38659,2.014284,25000,2021-11-15
1,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,23.38659,2.014284,25000,2021-11-16
2,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,23.38659,2.014284,25000,2021-11-17
3,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,23.38659,2.014284,25000,2021-11-18
4,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,23.38659,2.014284,25000,2021-11-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.189362,0.83875,16.094074,167.534717,48.793422,65.494271,23.997497,0.079903,81.360874,496.527876,34.84512,3.840225,39300,2022-01-29
76,0.189362,0.83875,16.094074,167.534717,48.793422,65.494271,23.997497,0.079903,81.360874,496.527876,34.84512,3.840225,39300,2022-01-30
77,0.189362,0.83875,16.094074,167.534717,48.793422,65.494271,23.997497,0.079903,81.360874,496.527876,34.84512,3.840225,39300,2022-01-31
78,0.189362,0.83875,16.094074,167.534717,48.793422,65.494271,23.997497,0.079903,81.360874,496.527876,34.84512,3.840225,39300,2022-02-01


# Collecting Price Data

Right now I am using Messari APIs to collect price information: https://messari.io/api/docs

I call the get_metric_timeseries endpoint to get price dataframes for every asset in the portfolio, specified by the asset list input.

The result is a dictionary (price_df) mapping asset names to a dataframe of its pricing information.

In [3]:
# Import Messari API wrapper
from messari.messari import Messari

# Set up Messari instance
MESSARI_API_KEY = 'e587edc0-9545-4a47-b4aa-64b9346e46cd'
messari = Messari(api_key=MESSARI_API_KEY)

price_dfs = {}

metric = 'price'      #Specifying we want asset prices to API
start = '2021-11-15'  #Starting Date of the LTG Portfolio
end = (datetime.now()).strftime('%Y-%m-%d')


#Connecting to messari.get_metric_timeseries to get timeseries price data
for x in assets:
    try:
        timeseries_df = messari.get_metric_timeseries(asset_slugs=x, asset_metric=metric, start=start, end=end)
        price_dfs[x] = timeseries_df
        time.sleep(10)
    except:
        time.sleep(10)

I unpack the prices to make a single dataframe containing daily open prices for each asset.

In [4]:
#Making an easy to read dataframe of prices        
price_df = pd.DataFrame(price_dfs["BTC"]["BTC"]['open'])
price_df = price_df.drop('open', axis = 1)

for x in price_dfs.keys():
    h = pd.DataFrame(price_dfs[x][x]['open'])
    h[x] = h['open']
    h = h.drop('open', axis = 1)
    
    price_df = pd.merge(price_df, h, left_index = True, right_index = True)
    
price_df = price_df.reset_index()
price_df["timestamp"] = price_df["timestamp"].astype(str)

In [5]:
price_df

,timestamp,BTC,ETH,SOL,UNI,AR,DOT,AVAX,YFI,LINK,CRV,LUNA,AAVE
0,2021-11-15,65486.265990,4627.851244,238.520690,24.572193,70.271528,46.409847,95.469040,33545.179098,33.739979,4.130946,51.111188,315.523348
1,2021-11-16,63566.456496,4562.620873,237.739806,23.968918,65.674945,44.738853,93.659583,33415.822371,32.031404,3.929542,48.130843,301.685877
2,2021-11-17,60122.200185,4212.800854,219.161277,21.699274,60.727527,40.869853,97.826484,30685.330613,29.192501,3.703333,43.255748,278.372865
3,2021-11-18,60377.770437,4288.432322,218.759079,21.880991,56.903782,42.749524,107.847419,31172.464743,29.203862,4.033781,43.205135,284.248778
4,2021-11-19,56959.758772,4002.838852,195.542443,20.100971,53.477628,39.072304,100.377932,32457.401721,26.552862,4.137283,40.303407,262.811146
5,2021-11-20,58126.153201,4299.319676,215.296268,21.221258,59.689161,41.272814,112.081241,31814.270563,28.302296,3.966698,42.433407,279.473282
6,2021-11-21,59750.205862,4413.549140,218.103256,22.032926,65.244938,42.114851,125.944266,31998.213337,28.357544,3.944000,45.213528,274.393628
7,2021-11-22,58750.755392,4271.806979,231.331586,21.694834,64.427821,41.998549,128.684396,31627.285012,29.039215,3.802926,43.610264,267.129997
8,2021-11-23,56292.980872,4088.618805,215.824084,20.967019,59.891996,39.495318,134.931615,31170.717771,26.763640,4.259820,41.105046,263.359081
9,2021-11-24,57564.149918,4341.315289,221.863011,22.172841,60.784098,40.775402,124.022085,31635.961405,26.925922,5.065807,42.711800,269.899043


# Portfolio Performance

To evaluate performance I compare returns versus invested capital for the LTG portfolio versus BTC returns over the same time period.

To start the comparison I load the historical LTG allocations and invested capital (from initial inputs) and merge with the prices df.

In [6]:
#Load CS Portfolio Data
allocations_df = pd.read_csv(cs_weights_csv)    #Loaded from original inputs
allocations_df["timestamp"] = allocations_df["Date"]
allocations_df["timestamp"] = allocations_df["timestamp"].astype(str)
allocations_df = allocations_df.drop('Date', axis = 1)

#Merge into a Portfolio and Price
cs_portfolio_df = pd.merge(allocations_df, price_df, how = 'left', on = 'timestamp', suffixes = ('_allocation', '_price'))

In [27]:
cs_portfolio_df

,BTC_allocation,ETH_allocation,SOL_allocation,UNI_allocation,AR_allocation,DOT_allocation,AVAX_allocation,YFI_allocation,LINK_allocation,CRV_allocation,...,LINK_Value,LINK_Weight,CRV_Value,CRV_Weight,LUNA_Value,LUNA_Weight,AAVE_Value,AAVE_Weight,CS_%,BTC_%
0,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1364.761961,0.054703,1237.574368,0.049605,1195.316391,0.047911,635.553533,0.025474,0.997947,1.000000
1,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1295.651117,0.053543,1177.236448,0.048650,1125.616302,0.046516,607.680942,0.025113,0.967931,0.970684
2,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1180.819220,0.052270,1109.467388,0.049112,1011.604440,0.044780,560.721921,0.024821,0.903631,0.918089
3,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1181.278757,0.051552,1208.465062,0.052739,1010.420786,0.044096,572.557678,0.024987,0.916564,0.921991
4,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1074.047405,0.049854,1239.473001,0.057533,942.559262,0.043751,529.376204,0.024572,0.861755,0.869797
5,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1144.810952,0.050638,1188.367926,0.052565,992.372690,0.043895,562.938473,0.024900,0.904307,0.887608
6,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1147.045690,0.049032,1181.568089,0.050507,1057.390246,0.045199,552.706610,0.023626,0.935760,0.912408
7,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1174.618899,0.050547,1139.304008,0.049027,1019.895354,0.043889,538.075597,0.023155,0.929527,0.897146
8,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1082.573222,0.048105,1276.183322,0.056708,961.306850,0.042716,530.479901,0.023572,0.900183,0.859615
9,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1089.137461,0.047022,1517.645884,0.065522,998.883353,0.043126,543.653240,0.023472,0.926489,0.879026


I find the historic value in USD by multiplying the allocation size by the price.  Adding all the values together gives a total portfolio value.

I also divide each asset value by the portfolio to get the asset weighting within the portfolio for each timestep.

Changes in portfolio value as a percentage can are found by dividing total portfolio value ("LTG_Value") by the total invested capital at each timestep.

In [7]:
#Find and Compare Portfolio Value to BTC
cs_portfolio_df['LTG_Value'] = ((cs_portfolio_df["BTC_allocation"] * cs_portfolio_df["BTC_price"]) +
                            (cs_portfolio_df["ETH_allocation"] * cs_portfolio_df["ETH_price"]) +
                            (cs_portfolio_df["SOL_allocation"] * cs_portfolio_df["SOL_price"]) +
                            (cs_portfolio_df["UNI_allocation"] * cs_portfolio_df["UNI_price"]) +
                            (cs_portfolio_df["AR_allocation"] * cs_portfolio_df["AR_price"]) +
                            (cs_portfolio_df["DOT_allocation"] * cs_portfolio_df["DOT_price"]) +
                            (cs_portfolio_df["AVAX_allocation"] * cs_portfolio_df["AVAX_price"]) +
                            (cs_portfolio_df["YFI_allocation"] * cs_portfolio_df["YFI_price"]) +
                            (cs_portfolio_df["LINK_allocation"] * cs_portfolio_df["LINK_price"]) +
                            (cs_portfolio_df["CRV_allocation"] * cs_portfolio_df["CRV_price"]) +
                            (cs_portfolio_df["LUNA_allocation"] * cs_portfolio_df["LUNA_price"]) +
                            (cs_portfolio_df["AAVE_allocation"] * cs_portfolio_df["AAVE_price"]))

cs_portfolio_df = cs_portfolio_df[cs_portfolio_df['LTG_Value'].notna()]

cs_portfolio_df["BTC_Value"] = (cs_portfolio_df["BTC_allocation"] * cs_portfolio_df["BTC_price"])
cs_portfolio_df["BTC_Weight"] = cs_portfolio_df["BTC_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["ETH_Value"] = (cs_portfolio_df["ETH_allocation"] * cs_portfolio_df["ETH_price"])
cs_portfolio_df["ETH_Weight"] = cs_portfolio_df["ETH_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["SOL_Value"] = (cs_portfolio_df["SOL_allocation"] * cs_portfolio_df["SOL_price"])
cs_portfolio_df["SOL_Weight"] = cs_portfolio_df["SOL_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["UNI_Value"] = (cs_portfolio_df["UNI_allocation"] * cs_portfolio_df["UNI_price"])
cs_portfolio_df["UNI_Weight"] = cs_portfolio_df["UNI_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["AR_Value"] = (cs_portfolio_df["AR_allocation"] * cs_portfolio_df["AR_price"])
cs_portfolio_df["AR_Weight"] = cs_portfolio_df["AR_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["DOT_Value"] = (cs_portfolio_df["DOT_allocation"] * cs_portfolio_df["DOT_price"])
cs_portfolio_df["DOT_Weight"] = cs_portfolio_df["DOT_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["AVAX_Value"] = (cs_portfolio_df["AVAX_allocation"] * cs_portfolio_df["AVAX_price"])
cs_portfolio_df["AVAX_Weight"] = cs_portfolio_df["AVAX_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["YFI_Value"] = (cs_portfolio_df["YFI_allocation"] * cs_portfolio_df["YFI_price"])
cs_portfolio_df["YFI_Weight"] = cs_portfolio_df["YFI_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["LINK_Value"] = (cs_portfolio_df["LINK_allocation"] * cs_portfolio_df["LINK_price"])
cs_portfolio_df["LINK_Weight"] = cs_portfolio_df["LINK_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["CRV_Value"] = (cs_portfolio_df["CRV_allocation"] * cs_portfolio_df["CRV_price"])
cs_portfolio_df["CRV_Weight"] = cs_portfolio_df["CRV_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["LUNA_Value"] = (cs_portfolio_df["LUNA_allocation"] * cs_portfolio_df["LUNA_price"])
cs_portfolio_df["LUNA_Weight"] = cs_portfolio_df["LUNA_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["AAVE_Value"] = (cs_portfolio_df["AAVE_allocation"] * cs_portfolio_df["AAVE_price"])
cs_portfolio_df["AAVE_Weight"] = cs_portfolio_df["AAVE_Value"] / cs_portfolio_df['LTG_Value']

cs_portfolio_df["CS_%"] = cs_portfolio_df['LTG_Value'] / cs_portfolio_df['Total Investment (USD)']


Finding the BTC return captures price movement in comparison to day zero.  Below is calcuating the cumulative return for BTC at each timestep.

In [8]:
#Calculate BTC Returns and Add to Dataframe
BTC_perc = []
for x in range(0, len(cs_portfolio_df["CS_%"])):
    daily_cumulative_return = price_df["BTC"].iloc[x] / price_df["BTC"].iloc[0]
    daily_cumulative_return = float(daily_cumulative_return)
    BTC_perc.append(daily_cumulative_return)
    
cs_portfolio_df["BTC_%"] = BTC_perc

cs_portfolio_df

,BTC_allocation,ETH_allocation,SOL_allocation,UNI_allocation,AR_allocation,DOT_allocation,AVAX_allocation,YFI_allocation,LINK_allocation,CRV_allocation,...,LINK_Value,LINK_Weight,CRV_Value,CRV_Weight,LUNA_Value,LUNA_Weight,AAVE_Value,AAVE_Weight,CS_%,BTC_%
0,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1364.761961,0.054703,1237.574368,0.049605,1195.316391,0.047911,635.553533,0.025474,0.997947,1.000000
1,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1295.651117,0.053543,1177.236448,0.048650,1125.616302,0.046516,607.680942,0.025113,0.967931,0.970684
2,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1180.819220,0.052270,1109.467388,0.049112,1011.604440,0.044780,560.721921,0.024821,0.903631,0.918089
3,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1181.278757,0.051552,1208.465062,0.052739,1010.420786,0.044096,572.557678,0.024987,0.916564,0.921991
4,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1074.047405,0.049854,1239.473001,0.057533,942.559262,0.043751,529.376204,0.024572,0.861755,0.869797
5,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1144.810952,0.050638,1188.367926,0.052565,992.372690,0.043895,562.938473,0.024900,0.904307,0.887608
6,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1147.045690,0.049032,1181.568089,0.050507,1057.390246,0.045199,552.706610,0.023626,0.935760,0.912408
7,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1174.618899,0.050547,1139.304008,0.049027,1019.895354,0.043889,538.075597,0.023155,0.929527,0.897146
8,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1082.573222,0.048105,1276.183322,0.056708,961.306850,0.042716,530.479901,0.023572,0.900183,0.859615
9,0.111606,0.50302,9.472038,91.625760,26.943127,33.958418,15.161893,0.043266,40.449402,299.586198,...,1089.137461,0.047022,1517.645884,0.065522,998.883353,0.043126,543.653240,0.023472,0.926489,0.879026


I repeat the above by on a 30 day window instead of the entire life of the portfolio.  There are some edge cases which could break this so I wouldn't suggest using it on the investments pages for now.

In [9]:
#Compare 30 Day Portfolio Value to 30 Day BTC Performance
BTC_Monthly = []
CS_Monthly = []
for x in range(-31, 0):
    cumulative_return = price_df["BTC"].iloc[x] / price_df["BTC"].iloc[-31]
    cumulative_return = float(cumulative_return)
    BTC_Monthly.append(cumulative_return)
    
    cumulative_return_cs = cs_portfolio_df['LTG_Value'].iloc[x] / cs_portfolio_df['LTG_Value'].iloc[-31]
    cumulative_return_cs = float(cumulative_return_cs)
    CS_Monthly.append(cumulative_return_cs)
    

monthly_comparison = pd.DataFrame(np.column_stack((cs_portfolio_df["timestamp"].iloc[-31:], CS_Monthly, BTC_Monthly)), columns = ["", "CS","BTC"])

monthly_comparison

,,CS,BTC
0,2021-12-13,1.0,1.0
1,2021-12-14,0.899711,0.933145
2,2021-12-15,0.941019,0.965867
3,2021-12-16,0.984399,0.975682
4,2021-12-17,0.986631,0.951148
5,2021-12-18,1.016545,0.921113
6,2021-12-19,1.054019,0.935193
7,2021-12-20,1.047044,0.932094
8,2021-12-21,1.056706,0.936108
9,2021-12-22,1.093236,0.976369


Total returns are found by subtracting cumulative returns (a variable number) from the returns at day 0 (always 1).

This is down for both the LTG portfolio (cs_total_return) and BTC (btc_total_return).

Outperformance (or under performance) in terms of percentage points is calculated by subtracting cs_total_return from btc_total_return.

In [10]:
#Find Total Returns
btc_total_return = cs_portfolio_df["BTC_%"].iloc[-1] - 1
cs_total_return = cs_portfolio_df["CS_%"].iloc[-1] - 1

btc_total_return = round(btc_total_return,4)
cs_total_return = round(cs_total_return,4)

#Find Monthly Returns
btc_month_return = (cs_portfolio_df["BTC_%"].iloc[-1] - cs_portfolio_df["BTC_%"].iloc[-31]) / cs_portfolio_df["BTC_%"].iloc[-31]
cs_month_return = (cs_portfolio_df["CS_%"].iloc[-1] - cs_portfolio_df["CS_%"].iloc[-31]) / cs_portfolio_df["CS_%"].iloc[-31]

btc_month_return = round(btc_month_return,4)
cs_month_return = round(cs_month_return,4)


#Convert to Strings
btc_string = "BTC had a Total Return of {}"
btc_string = btc_string.format(btc_total_return)

cs_string = "The CS Portfolio had a Total Return of {}"
cs_string = cs_string.format(cs_total_return)

btc_month_string = "BTC had a Monthly 30-day of {}"
btc_month_string = btc_month_string.format(btc_month_return)

cs_month_string = "The CS Portfolio had a 30-day Return of {}"
cs_month_string = cs_month_string.format(cs_month_return)


# Monthly changes in Weightings and Contributions

Monthly Changes in Asset Weightings and Monthly Return Contributions (Not needed for Investments Page)

In [11]:
#Month Weights and Contribution
asset = []
growth = []


for x in price_df[-31:]:
    if x == 'timestamp': pass
    else:
        per_return = ((price_df[x].iloc[-1] - price_df[x].iloc[-31]) / price_df[x].iloc[-31])
        asset.append(x)
        growth.append(round(per_return,4))
        
cs_returns_month = pd.DataFrame(np.column_stack((asset, growth)), columns = ["Asset", "Total_Return"])
cs_returns_month["Total_Return"] = cs_returns_month["Total_Return"].astype(float)

Starting_Weights = []
Starting_Weights.append(cs_portfolio_df["BTC_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["ETH_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["SOL_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["UNI_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["AR_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["DOT_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["AVAX_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["YFI_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["LINK_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["CRV_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["LUNA_Weight"].iloc[-31])
Starting_Weights.append(cs_portfolio_df["AAVE_Weight"].iloc[-31])

cs_returns_month["Starting_Weights"] = Starting_Weights
cs_returns_month["Total_Contribution"] = cs_returns_month["Total_Return"] * cs_returns_month["Starting_Weights"]

End_Weights = []
End_Weights.append(cs_portfolio_df["BTC_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["ETH_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["SOL_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["UNI_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["AR_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["DOT_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["AVAX_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["YFI_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["LINK_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["CRV_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["LUNA_Weight"].iloc[-1])
End_Weights.append(cs_portfolio_df["AAVE_Weight"].iloc[-1])

cs_returns_month["End_Weights"] = End_Weights
cs_returns_month["Percent_Change"] = (cs_returns_month["End_Weights"] - cs_returns_month["Starting_Weights"]) / cs_returns_month["Starting_Weights"]

cs_returns_month["Category"] = np.where(cs_returns_month["Asset"] == "BTC", "BTC", " ")
cs_returns_month["Category"] = np.where(cs_returns_month["Asset"].isin(["ETH", "SOL", "DOT", "AVAX", "LUNA"]), "L1", cs_returns_month["Category"])
cs_returns_month["Category"] = np.where(cs_returns_month["Asset"].isin(["UNI", "YFI", "CRV", "AAVE"]), "DeFi", cs_returns_month["Category"])
cs_returns_month["Category"] = np.where(cs_returns_month["Asset"].isin(["LINK", "AR"]), "Data", cs_returns_month["Category"])

category_breakdown_month = cs_returns_month.groupby("Category")["End_Weights"].sum()

category_returns_month = cs_returns_month.groupby("Category")["Total_Contribution"].sum()


# Portfolio Characteristics

Portfolio Characteristics (Strongly suggest adding to investment page)

Characteristics:
- Portfolio and BTC Standard Variation (traditional measurment of risk)

- Beta Values (compared to BTC) another measure of risk and volatility

- Correlation Coefficient with BTC

In [12]:
#Total Portfolio Characteristics
CS_Daily_Return = []
BTC_Daily_Return = []

for x in range(0,len(cs_portfolio_df)):
    if x == 0:
        LTG_difference = 0
        BTC_difference = 0
    else:
        LTG_difference = cs_portfolio_df["CS_%"].iloc[x] - cs_portfolio_df["CS_%"].iloc[x-1]
        BTC_difference = cs_portfolio_df["BTC_%"].iloc[x] - cs_portfolio_df["BTC_%"].iloc[x-1]
        
    CS_Daily_Return.append(LTG_difference)
    BTC_Daily_Return.append(BTC_difference)

daily_returns = pd.DataFrame(np.column_stack((cs_portfolio_df["timestamp"], CS_Daily_Return, BTC_Daily_Return)), columns = ["timestamp", "CS_daily", "BTC_daily"])

daily_returns["CS_daily"] = daily_returns["CS_daily"].astype(float)
daily_returns["BTC_daily"] = daily_returns["BTC_daily"].astype(float)


Standard Deviation Calculations


In [13]:
daily_return_CS = round(daily_returns["CS_daily"].mean(), 4)
daily_std_CS = round(daily_returns["CS_daily"].std(), 4)

daily_return_BTC = round(daily_returns["BTC_daily"].mean(), 4)
daily_std_BTC = round(daily_returns["BTC_daily"].std(), 4)

Beta Calculation (https://www.investopedia.com/ask/answers/070615/what-formula-calculating-beta.asp)

and Correlation Coefficient (https://www.investopedia.com/ask/answers/032515/what-does-it-mean-if-correlation-coefficient-positive-negative-or-zero.asp)

In [14]:
btc_var =  np.var(daily_returns["BTC_daily"])   #BTC Variance for Beta
cov_matrix = daily_returns.cov()                #Covariance Matrix for Finding Beta

cov_cs = cov_matrix.iloc[1][1]
coe_cs = round((cov_cs / (daily_std_CS * daily_std_BTC)),4)    #Correlation Coefficient for BTC vs LTG
CS_B = cov_cs/btc_var                                          #BTC based Beta for LTG
CS_B = round(CS_B,4)

data_names = ["Daily Return", "Standard Deviation", "BTC_Beta", "BTC_Correlation"]
CS = [daily_return_CS, daily_std_CS, CS_B, coe_cs]
BTC = [daily_return_BTC, daily_std_BTC, 1, 1]

portfolio_characteristics = pd.DataFrame(np.column_stack((data_names, CS, BTC)), columns = ["", "CS","BTC"])

portfolio_characteristics

,,CS,BTC
0,Daily Return,-0.0029,-0.0059
1,Standard Deviation,0.0358,0.0259
2,BTC_Beta,1.0172,1.0
3,BTC_Correlation,0.7261,1.0


Repeat for Monthly

In [15]:
#Monthly Portfolio Characteristics
daily_returns = pd.DataFrame(np.column_stack((cs_portfolio_df["timestamp"].iloc[-31:], CS_Daily_Return[-31:], BTC_Daily_Return[-31:])), columns = ["timestamp", "CS_daily", "BTC_daily"])

daily_returns["CS_daily"] = daily_returns["CS_daily"].astype(float)
daily_returns["BTC_daily"] = daily_returns["BTC_daily"].astype(float)

daily_return_CS = round(daily_returns["CS_daily"].mean(), 4)
daily_std_CS = round(daily_returns["CS_daily"].std(), 4)

daily_return_BTC = round(daily_returns["BTC_daily"].mean(), 4)
daily_std_BTC = round(daily_returns["BTC_daily"].std(), 4)

btc_var =  np.var(daily_returns["BTC_daily"])
cov_matrix = daily_returns.cov()

cov_cs = cov_matrix.iloc[0][1]
coe_cs = round((cov_cs / (daily_std_CS * daily_std_BTC)),4)
CS_B = cov_cs/btc_var
CS_B = round(CS_B,4)

data_names = ["Daily Return", "Standard Deviation", "BTC_Beta", "BTC_Correlation"]
CS = [daily_return_CS, daily_std_CS, CS_B, coe_cs]
BTC = [daily_return_BTC, daily_std_BTC, 1, 1]

portfolio_month_characteristics = pd.DataFrame(np.column_stack((data_names, CS, BTC)), columns = ["", "CS","BTC"])

portfolio_month_characteristics

,,CS,BTC
0,Daily Return,0.0002,-0.0033
1,Standard Deviation,0.0342,0.0202
2,BTC_Beta,1.517,1.0
3,BTC_Correlation,0.8638,1.0


# Outputs

In [16]:
# Total Returns (%)
cs_portfolio_df[["timestamp", "CS_%", "BTC_%"]]

,timestamp,CS_%,BTC_%
0,2021-11-15,0.997947,1.000000
1,2021-11-16,0.967931,0.970684
2,2021-11-17,0.903631,0.918089
3,2021-11-18,0.916564,0.921991
4,2021-11-19,0.861755,0.869797
5,2021-11-20,0.904307,0.887608
6,2021-11-21,0.935760,0.912408
7,2021-11-22,0.929527,0.897146
8,2021-11-23,0.900183,0.859615
9,2021-11-24,0.926489,0.879026


In [17]:
# Comparing Characteristics
portfolio_characteristics

,,CS,BTC
0,Daily Return,-0.0029,-0.0059
1,Standard Deviation,0.0358,0.0259
2,BTC_Beta,1.0172,1.0
3,BTC_Correlation,0.7261,1.0


In [18]:
#Total Returns
print(btc_string)
print(cs_string)

BTC had a Total Return of -0.3472
The CS Portfolio had a Total Return of -0.1755


## Monthly Data

In [19]:
monthly_comparison

,,CS,BTC
0,2021-12-13,1.0,1.0
1,2021-12-14,0.899711,0.933145
2,2021-12-15,0.941019,0.965867
3,2021-12-16,0.984399,0.975682
4,2021-12-17,0.986631,0.951148
5,2021-12-18,1.016545,0.921113
6,2021-12-19,1.054019,0.935193
7,2021-12-20,1.047044,0.932094
8,2021-12-21,1.056706,0.936108
9,2021-12-22,1.093236,0.976369


In [20]:
print(btc_month_string)
print(cs_month_string)

BTC had a Monthly 30-day of -0.1467
The CS Portfolio had a 30-day Return of -0.0084


In [21]:
portfolio_month_characteristics

,,CS,BTC
0,Daily Return,0.0002,-0.0033
1,Standard Deviation,0.0342,0.0202
2,BTC_Beta,1.517,1.0
3,BTC_Correlation,0.8638,1.0


In [22]:
cs_returns_month

,Asset,Total_Return,Starting_Weights,Total_Contribution,End_Weights,Percent_Change,Category
0,BTC,-0.1467,0.290306,-0.042588,0.249819,-0.139463,BTC
1,ETH,-0.2161,0.106119,-0.022932,0.083892,-0.209453,L1
2,SOL,-0.1854,0.085300,-0.015815,0.070074,-0.178502,L1
3,UNI,-0.0025,0.082104,-0.000205,0.082595,0.005981,DeFi
4,AR,0.1554,0.062386,0.009695,0.072694,0.165231,Data
5,DOT,-0.1360,0.059315,-0.008067,0.051682,-0.128685,L1
6,AVAX,0.0168,0.064712,0.001087,0.066362,0.025499,L1
7,YFI,0.5056,0.053194,0.026895,0.080773,0.518449,DeFi
8,LINK,0.3028,0.050922,0.015419,0.066907,0.313892,Data
9,CRV,0.2005,0.058395,0.011708,0.070699,0.210691,DeFi


In [23]:
category_breakdown_month

Category
BTC     0.249819
Data    0.139601
DeFi    0.259350
L1      0.351230
Name: End_Weights, dtype: float64

In [24]:
category_returns_month

Category
BTC    -0.042588
Data    0.025114
DeFi    0.042323
L1     -0.033274
Name: Total_Contribution, dtype: float64